In [1]:
import math
import pandas as pd
import numpy as np
from collections import Counter
from random import seed
import os
import traceback
import json
from enum import Enum

In [2]:
df_ies_prom = pd.read_csv("../DataFiles/ies_prom_col_1_2.csv", encoding="latin-1")
df_ies_sing = pd.read_csv('../DataFiles/ies_prom_singulares.csv', encoding="latin-1")
df_candidaturas = pd.read_csv("../DataFiles/Geral_candidaturas.csv", encoding="latin-1")
df_anulacoes = pd.read_csv("../DataFiles/Anulações-Resposta.csv", encoding="latin-1")
df_facie_appie = pd.read_csv("../DataFiles/APPIeFACIE-Resposta.csv", encoding="latin-1")

In [3]:
"""
Excluir projetos repetidos para o mesmo ano
"""
df_ies_prom['rank'] = df_ies_prom.groupby(['NIF_anon', 'ANO_EXERCICIO'])['DATA_RECEPCAO'].rank(method='first', ascending=False)
df_ies_prom['rank'] = pd.to_numeric(df_ies_prom['rank'], downcast='integer')
df_ies_prom_clean = df_ies_prom[df_ies_prom['rank']==1]

In [4]:
df_ies_prom.shape

(123429, 214)

In [5]:
"""
foram retirados 6,354 registos de projetos para o mesmo ANO_EXERCICIO (os projetos com DATA_RECEPCAO mais recente prevalece)
"""
df_ies_prom_clean.shape

(117075, 214)

In [6]:
path_save = "../Output/"

In [7]:
"""
Valida se conjunto de dados tem projetos para o mesmo ano
"""
def check_project_year_candidate_duplicates(df):
    new_df = df.groupby(['N_Proj_anon', 'ANO_EXERCICIO']).size()
    new_df = new_df.to_frame(name = 'size').reset_index()
    display(new_df[new_df['size']>1])

In [8]:
display(df_ies_prom_clean)

,NIF_anon,ANO_EXERCICIO,DATA_RECEPCAO,_5001_VENDAS_SERVICOS_PRESTADOS,_5002_SUBSIDIOS_EXPLORACAO,_5003_GANHOS_PERDAS_SUBSIDIARIAS,_5004_VARIA_INVENTARIOS_PRODUCAO,_5005_TRABALHOS_PROPRIA_ENTIDADE,_5006_CMVMC,_5007_FSE,...,_6078_1_IMG_INT_PM_VEND_PREST_SERV,_6078_2_IMG_COM_PM_VEND_PREST_SERV,_6078_3_IMG_EC_PM_VEND_PREST_SERV,_6078_4_IMG_TOT_PM_VEND_PREST_SERV,_6079_1_IMG_INT_PM_COMP_FORN_SER_EXT,_6079_2_IMG_COM_PM_COMP_FORN_SER_EXT,_6079_3_IMG_EC_PM_COMP_FORN_SER_EXT,_6079_4_IMG_TOT_PM_COMP_FORN_SER_EXT,CAE_SUBCLASSE,rank
0,333983332,2013,58:36.0,206126.13,0.00,0.0,203983.1,0.0,184300.39,44817.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46140.0,1
1,333983332,2014,04:40.0,286608.03,0.00,0.0,196804.9,0.0,306921.32,44750.45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46140.0,1
2,333983332,2015,17:57.0,333328.69,0.00,0.0,-20659.0,0.0,143963.59,43842.14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46140.0,1
3,333983332,2016,52:58.0,244459.14,0.00,0.0,-102172.0,0.0,97553.83,39402.56,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46140.0,1
4,333983332,2017,56:59.0,398805.99,0.00,0.0,-110024.0,0.0,152453.55,34936.46,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46140.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123422,938122144,2018,55:19.0,10831029.21,0.00,0.0,0.0,0.0,0.00,9800078.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62090.0,1
123423,938122144,2019,15:41.0,34216636.27,0.00,0.0,0.0,0.0,0.00,31128348.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62090.0,1
123424,639327193,2019,55:59.0,50000.00,3429.08,0.0,0.0,0.0,0.00,42636.48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1
123427,639327193,2020,11:25.0,47957.91,6830.42,0.0,0.0,0.0,613.94,28813.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10320.0,1


### Merge com ficheiro candidaturas

In [9]:
print("IES")
print("Número de nifs únicos: ", len(df_ies_prom_clean['NIF_anon'].unique()))
print("Existe nifs a null: ", df_ies_prom_clean['NIF_anon'].isnull().values.any())
print("Número nifs a null", df_ies_prom_clean['NIF_anon'].isnull().sum())

IES
Número de nifs únicos:  16597
Existe nifs a null:  False
Número nifs a null 0


In [10]:
df_ies_prom_candidaturas = df_ies_prom_clean.merge(df_candidaturas, left_on='NIF_anon', right_on='NIF_Prom_anon', how='inner')

In [11]:
df_ies_prom_candidaturas.shape

(44627, 403)

In [12]:
df_ies_prom_candidaturas.loc[:,['N_Proj_anon']].drop_duplicates(subset="N_Proj_anon")

,N_Proj_anon
0,21149
8,5582921
16,607793
24,5580038
32,465007
...,...
44621,3545539
44622,3546500
44623,3544361
44625,3546686


### Merge com projetos de empresas singulares

In [13]:
df_ies_candidaturas_renamed = df_ies_prom_candidaturas.rename(
columns={
'_5001_VENDAS_SERVICOS_PRESTADOS': 'VENDAS_SERVICOS_PRESTADOS',
'_5002_SUBSIDIOS_EXPLORACAO': 'SUBSIDIOS_EXPLORACAO',
'_5003_GANHOS_PERDAS_SUBSIDIARIAS': 'GANHOS_PERDAS_SUBSIDIARIAS',
'_5004_VARIA_INVENTARIOS_PRODUCAO': 'VARIA_INVENTARIOS_PRODUCAO',
'_5005_TRABALHOS_PROPRIA_ENTIDADE': 'TRABALHOS_PROPRIA_ENTIDADE',
'_5006_CMVMC': 'CMVMC',
'_5007_FSE': 'FSE',
'_5008_GASTOS_PESSOAL': 'GASTOS_PESSOAL',
'_5009_IMPARIDADE_INVENTARIOS': 'IMPARIDADE_INVENTARIOS',
'_5010_IMPARIDADE_DIVIDAS_A_RECEBER': 'IMPARIDADE_DIVIDAS_A_RECEBER',
'_5011_PROVISOES': 'PROVISOES',
'_5012_IMPARIDADE_INVENTA_N_AMORT_INVESTIMENTOS_N_DEPRECIAVEIS': 'IMPARIDADE_INVENTA_N_AMORT_INVESTIMENTOS_N_DEPRECIAVEIS',
'_5013_OUTRAS_IMPARIDADES': 'OUTRAS_IMPARIDADES',
'_5014_AUMENTOS_RED_JUSTO_VALOR': 'AUMENTOS_RED_JUSTO_VALOR',
'_5015_OUTROS_REDIMENTOS_GANHOS': 'OUTROS_REDIMENTOS_GANHOS',
'_5016_OUTROS_GASTOS_PERDAS': 'OUTROS_GASTOS_PERDAS',
'_5017_RES_ANTES_DEPRECIACAO_GASTOS': 'RES_ANTES_DEPRECIACAO_GASTOS',
'_5018_GASTOS_DEPRECIACAO_AMORTIZA': 'GASTOS_DEPRECIACAO_AMORTIZA',
'_5019_IMPARIDADE_INVEST_AMORTIZ_DEPRECIAVEIS': 'IMPARIDADE_INVEST_AMORTIZ_DEPRECIAVEIS',
'_5020_RESULTADO_OPERACIONAL': 'RESULTADO_OPERACIONAL',
'_5021_JUROS_RENDIME_SIMILARES_OBT': 'JUROS_RENDIME_SIMILARES_OBT',
'_5022_JUROS_GAST_SIMILARES_SUPORT': 'JUROS_GAST_SIMILARES_SUPORT',
'_5023_RES_ANTES_IMPOSTOS': 'RES_ANTES_IMPOSTOS',
'_5024_IMPOSTO_RENDIMENTO_PERIODO': 'IMPOSTO_RENDIMENTO_PERIODO',
'_5025_RESULTADO_LIQUIDO_PERIODO': 'RESULTADO_LIQUIDO_PERIODO',
'_5101_ATIVO_NCOR_FIXOS_TANGIVEIS': 'ATIVO_NCOR_FIXOS_TANGIVEIS',
'_5102_ATIVO_NCOR_PROPRI_INVESTIMENTO': 'ATIVO_NCOR_PROPRI_INVESTIMENTO',
'_5103_ATIVO_NCOR_GOODWILL': 'ATIVO_NCOR_GOODWILL',
'_5104_ATIVO_NCOR_INTANGIVEIS': 'ATIVO_NCOR_INTANGIVEIS',
'_5105_ATIVO_NCOR_BIOLOGICOS': 'ATIVO_NCOR_BIOLOGICOS',
'_5106_ATIVO_NCOR_PART_FINAN_EQV_PAT': 'ATIVO_NCOR_PART_FINAN_EQV_PAT',
'_5107_ATIVO_NCOR_PART_FIN_OUTROS_MET': 'ATIVO_NCOR_PART_FIN_OUTROS_MET',
'_5108_ATIVO_NCOR_ACCIONISTAS_SOCIOS': 'ATIVO_NCOR_ACCIONISTAS_SOCIOS',
'_5109_ATIVO_NCOR_OUTROS_ACT_FINANC': 'ATIVO_NCOR_OUTROS_ACT_FINANC',
'_5110_ATIVO_NCOR_IMPOSTOS_DIFERIDOS': 'ATIVO_NCOR_IMPOSTOS_DIFERIDOS',
'_5111_ATIVO_NCOR_INV_FINANC_PQ_ENTID': 'ATIVO_NCOR_INV_FINANC_PQ_ENTID',
'_5112_ATIVO_NCOR_TOTAL': 'ATIVO_NCOR_TOTAL',
'_5113_ATIVO_COR_INVENTARIOS': 'ATIVO_COR_INVENTARIOS',
'_5114_ATIVO_COR_ACTIVOS_BIOLOGICOS': 'ATIVO_COR_ACTIVOS_BIOLOGICOS',
'_5115_ATIVO_COR_CLIENTES': 'ATIVO_COR_CLIENTES',
'_5116_ATIVO_COR_ADIANTAMENTOS_FORNEC': 'ATIVO_COR_ADIANTAMENTOS_FORNEC',
'_5117_ATIVO_COR_ESTADO_OUT_ENTES_PUB': 'ATIVO_COR_ESTADO_OUT_ENTES_PUB',
'_5118_ATIVO_COR_ACCIONISTAS_SOCIOS': 'ATIVO_COR_ACCIONISTAS_SOCIOS',
'_5119_ATIVO_COR_OUTRA_CONT_A_RECEBER': 'ATIVO_COR_OUTRA_CONT_A_RECEBER',
'_5120_ATIVO_COR_DIFERIMENTOS': 'ATIVO_COR_DIFERIMENTOS',
'_5121_ATIVO_COR_ACT_FINAC_DETIDO_NEG': 'ATIVO_COR_ACT_FINAC_DETIDO_NEG',
'_5122_ATIVO_COR_OUTROS_ACT_FINANCEIR': 'ATIVO_COR_OUTROS_ACT_FINANCEIR',
'_5123_ATIVO_COR_ACT_N_COR_DET_VENDA': 'ATIVO_COR_ACT_N_COR_DET_VENDA',
'_5124_ATIVO_COR_OUTROS_ACT_CORRENTES': 'ATIVO_COR_OUTROS_ACT_CORRENTES',
'_5125_ATIVO_COR_CAIXA_DEP_BANCARIOS': 'ATIVO_COR_CAIXA_DEP_BANCARIOS',
'_5126_ATIVO_COR_TOTAL': 'ATIVO_COR_TOTAL',
'_5127_ATIVO_TOTAL': 'ATIVO_TOTAL',
'_5128_CP_CAPITAL_REALIZADO': 'CP_CAPITAL_REALIZADO',
'_5129_CP_ACCOES_PROPRIAS': 'CP_ACCOES_PROPRIAS',
'_5130_CP_OUTROS_INSTR_CAP_PROPRIO': 'CP_OUTROS_INSTR_CAP_PROPRIO',
'_5131_CP_PREMIOS_EMISSAO': 'CP_PREMIOS_EMISSAO',
'_5132_CP_RESERVAS_LEGAIS': 'CP_RESERVAS_LEGAIS',
'_5133_CP_OUTRAS_RESERVAS': 'CP_OUTRAS_RESERVAS',
'_5134_CP_RESULTADOS_TRANSITADOS': 'CP_RESULTADOS_TRANSITADOS',
'_5135_CP_AJUST_EM_ACT_FINANCEIROS': 'CP_AJUST_EM_ACT_FINANCEIROS',
'_5136_CP_EXCENDENTES_REVALORIZACAO': 'CP_EXCENDENTES_REVALORIZACAO',
'_5137_CP_OUTRAS_VARIACAOES_CAP_PRO': 'CP_OUTRAS_VARIACAOES_CAP_PRO',
'_5138_CP_SOMA': 'CP_SOMA',
'_5139_CP_RESULTADO_LIQUIDO_PERIODO': 'CP_RESULTADO_LIQUIDO_PERIODO',
'_5140_CP_DIVIDENDOS_ANTECIPADOS': 'CP_DIVIDENDOS_ANTECIPADOS',
'_5141_CP_TOTAL': 'CP_TOTAL',
'_5142_PASSIVO_NC_PROVISOES': 'PASSIVO_NC_PROVISOES',
'_5143_PASSIVO_NC_FINANCIAMENTOS_OBTD': 'PASSIVO_NC_FINANCIAMENTOS_OBTD',
'_5144_PASSIVO_NC_RESP_BENEF_POS_EMPREG': 'PASSIVO_NC_RESP_BENEF_POS_EMPREG',
'_5145_PASSIVO_NC_IMPOSTOS_DIFERIDOS': 'PASSIVO_NC_IMPOSTOS_DIFERIDOS',
'_5146_PASSIVO_NC_OUTRAS_CONTAS_A_PAGAR': 'PASSIVO_NC_OUTRAS_CONTAS_A_PAGAR',
'_5147_PASSIVO_NC_TOTAL': 'PASSIVO_NC_TOTAL',
'_5148_PASSIVO_COR_FORNCEDORES': 'PASSIVO_COR_FORNCEDORES',
'_5149_PASSIVO_COR_ADIANTA_DE_CLIENTES': 'PASSIVO_COR_ADIANTA_DE_CLIENTES',
'_5150_PASSIVO_COR_ESTADO_OUT_ENT_PUB': 'PASSIVO_COR_ESTADO_OUT_ENT_PUB',
'_5151_PASSIVO_COR_ACCIONISTAS_SOCIOS': 'PASSIVO_COR_ACCIONISTAS_SOCIOS',
'_5152_PASSIVO_COR_FINANCIAMENTOS_OBTD': 'PASSIVO_COR_FINANCIAMENTOS_OBTD',
'_5153_PASSIVO_COR_OUT_CONTAS_A_PAGAR': 'PASSIVO_COR_OUT_CONTAS_A_PAGAR',
'_5154_PASSIVO_COR_DIFERIMENTOS': 'PASSIVO_COR_DIFERIMENTOS',
'_5155_PASSIVO_COR_PASS_FINANC_DET_NEG': 'PASSIVO_COR_PASS_FINANC_DET_NEG',
'_5156_PASSIVO_COR_OUTROS_PASS_FINANCEI': 'PASSIVO_COR_OUTROS_PASS_FINANCEI',
'_5157_PASSIVO_COR_PAS_NC_DETIDOS_VENDA': 'PASSIVO_COR_PAS_NC_DETIDOS_VENDA',
'_5158_PASSIVO_COR_OUTROS_PAS_CORRENTES': 'PASSIVO_COR_OUTROS_PAS_CORRENTES',
'_5159_PASSIVO_COR_TOTAL': 'PASSIVO_COR_TOTAL',
'_5160_PASSIVO_TOTAL': 'PASSIVO_TOTAL',
'_5161_CP_E_PASSIVO_TOTAL': 'CP_E_PASSIVO_TOTAL',
'_6012_1_PESSOAL_NMP_PSERNR': '1_PESSOAL_NMP_PSERNR',
'_6012_2_PESSOAL_NHT_PSERNR': '2_PESSOAL_NHT_PSERNR',
'_6013_1_PESSOAL_NMP_PESS_REMUN_SE': '1_PESSOAL_NMP_PESS_REMUN_SE',
'_6013_2_PESSOAL_NHT_PESS_REMUN_SE': '2_PESSOAL_NHT_PESS_REMUN_SE',
'_6014_1_PESSOAL_NMP_PESS_N_REMUN': '1_PESSOAL_NMP_PESS_N_REMUN',
'_6014_2_PESSOAL_NHT_PESS_N_REMUN': '2_PESSOAL_NHT_PESS_N_REMUN',
'_6015_1_PESSOAL_NMP_PSE_TEMPO_COMPLETO': '1_PESSOAL_NMP_PSE_TEMPO_COMPLETO',
'_6015_2_PESSOAL_NHT_PSE_TEMPO_COMPLETO': '2_PESSOAL_NHT_PSE_TEMPO_COMPLETO',
'_6016_1_PESSOAL_NMP_PSETC_REMUNERADAS': '1_PESSOAL_NMP_PSETC_REMUNERADAS',
'_6016_2_PESSOAL_NHT_PSETC_REMUNERADAS': '2_PESSOAL_NHT_PSETC_REMUNERADAS',
'_6017_1_PESSOAL_NMP_PSE_TEMPO_PARCIAL': '1_PESSOAL_NMP_PSE_TEMPO_PARCIAL',
'_6017_2_PESSOAL_NHT_PSE_TEMPO_PARCIAL': '2_PESSOAL_NHT_PSE_TEMPO_PARCIAL',
'_6018_1_PESSOAL_NMP_PSETP_REMUNERADAS': '1_PESSOAL_NMP_PSETP_REMUNERADAS',
'_6018_2_N_PESSOAL_NHT_PSETP_REMUNERADAS': '2_N_PESSOAL_NHT_PSETP_REMUNERADAS',
'_6019_1_PESSOAL_NMP_PSE_HOMENS': '1_PESSOAL_NMP_PSE_HOMENS',
'_6019_2_PESSOAL_NHT_PSE_HOMENS': '2_PESSOAL_NHT_PSE_HOMENS',
'_6020_1_PESSOAL_NMP_PSE_MULHERES': '1_PESSOAL_NMP_PSE_MULHERES',
'_6020_2_PESSOAL_NHT_PSE_MULHERES': '2_PESSOAL_NHT_PSE_MULHERES',
'_6021_PESSOAL_NMP_PSE_INVEST_DESENV': 'PESSOAL_NMP_PSE_INVEST_DESENV',
'_6022_1_PESSOAL_NHT_PRESTAD_SERVICOS': '1_PESSOAL_NHT_PRESTAD_SERVICOS',
'_6022_2_PESSOAL_NMP_PRESTAD_SERVICOS': '2_PESSOAL_NMP_PRESTAD_SERVICOS',
'_6023_PESSOAL_NHT_PESS_AGENC_TEMPOR': 'PESSOAL_NHT_PESS_AGENC_TEMPOR',
'_6024_GASTOS_PESSOAL_TOTAL': 'GASTOS_PESSOAL_TOTAL',
'_6025_GP_REMUN_ORGAOS_SOCIAIS': 'GP_REMUN_ORGAOS_SOCIAIS',
'_6026_GP_ROS_PARTIC_LUCROS': 'GP_ROS_PARTIC_LUCROS',
'_6027_GP_REMUN_PESSOAL': 'GP_REMUN_PESSOAL',
'_6028_GP_RP_PARTIC_LUCROS': 'GP_RP_PARTIC_LUCROS',
'_6029_BENEF_POS_EMPREG': 'BENEF_POS_EMPREG',
'_6030_GP_BPE_PREMIOS_PENSOES': 'GP_BPE_PREMIOS_PENSOES',
'_6031_GP_BPE_OUTROS_BENEF': 'GP_BPE_OUTROS_BENEF',
'_6032_GP_BPE_PLAN_CONTRIB_ORG_SOC': 'GP_BPE_PLAN_CONTRIB_ORG_SOC',
'_6033_GP_BPE_PLAN_CONTRIB_OUTROS': 'GP_BPE_PLAN_CONTRIB_OUTROS',
'_6034_GP_INDEMINIZACOES': 'GP_INDEMINIZACOES',
'_6035_GP_ENCARG_REMUN': 'GP_ENCARG_REMUN',
'_6036_GP_SEG_ACID_TRAB_DOEN_PROF': 'GP_SEG_ACID_TRAB_DOEN_PROF',
'_6037_GP_GASTOS_ACCAO_SOCIAL': 'GP_GASTOS_ACCAO_SOCIAL',
'_6038_OUTROS_GASTOS_PESSOAL': 'OUTROS_GASTOS_PESSOAL',
'_6039_OGP_GASTOS_FORMACAO': 'OGP_GASTOS_FORMACAO',
'_6040_OGP_GASTOS_FARDAMENTO': 'OGP_GASTOS_FARDAMENTO',
'_6041_IAE_VENDAS': 'IAE_VENDAS',
'_6042_IAE_VENDAS_MERCADORIAS': 'IAE_VENDAS_MERCADORIAS',
'_6043_IAE_VENDAS_PAISDRR': 'IAE_VENDAS_PAISDRR',
'_6044_IAE_VENDAS_ACT_BIOL': 'IAE_VENDAS_ACT_BIOL',
'_6045_IAE_PREST_SERV': 'IAE_PREST_SERV',
'_6046_IAE_COMPRAS': 'IAE_COMPRAS',
'_6047_IAE_FORN_SERV_EXTER': 'IAE_FORN_SERV_EXTER',
'_6048_IAE_CMVMC': 'IAE_CMVMC',
'_6049_IAE_CMVMC_MERCADORIAS': 'IAE_CMVMC_MERCADORIAS',
'_6050_IAE_CMVMC_MATER_PRIMAS': 'IAE_CMVMC_MATER_PRIMAS',
'_6051_IAE_CMVMC_ACT_BIOL': 'IAE_CMVMC_ACT_BIOL',
'_6052_IAE_VAR_INVENT_PROD': 'IAE_VAR_INVENT_PROD',
'_6053_IAE_NUM_MED_PESS_SERV': 'IAE_NUM_MED_PESS_SERV',
'_6054_IAE_GASTOS_PESS': 'IAE_GASTOS_PESS',
'_6055_IAE_GASTOS_PESS_REMUN': 'IAE_GASTOS_PESS_REMUN',
'_6056_IAE_GASTOS_PESS_OUTROS': 'IAE_GASTOS_PESS_OUTROS',
'_6057_IAE_AFT_QUANT_ESCR_LIQ_FIN': 'IAE_AFT_QUANT_ESCR_LIQ_FIN',
'_6058_IAE_AFT_TOTAL_AQUIS': 'IAE_AFT_TOTAL_AQUIS',
'_6059_IAE_AFT_TOTAL_AQUIS_EDIF': 'IAE_AFT_TOTAL_AQUIS_EDIF',
'_6060_IAE_AFT_ADIC_PERIOD_ACT': 'IAE_AFT_ADIC_PERIOD_ACT',
'_6061_IAE_PI_QUANT_ESCR_LIQ_FIN': 'IAE_PI_QUANT_ESCR_LIQ_FIN',
'_6062_IAE_PI_TOTAL_AQUIS': 'IAE_PI_TOTAL_AQUIS',
'_6063_IAE_PI_TOTAL_AQUIS_EDIF': 'IAE_PI_TOTAL_AQUIS_EDIF',
'_6064_IAE_PI_ADIC_PERIO_PROP_INV': 'IAE_PI_ADIC_PERIO_PROP_INV',
'_6065_1_IMG_INT_VENDAS': '1_IMG_INT_VENDAS',
'_6065_2_IMG_COM_VENDAS': '2_IMG_COM_VENDAS',
'_6065_3_IMG_EC_VENDAS': '3_IMG_EC_VENDAS',
'_6065_4_IMG_TOT_VENDAS': '4_IMG_TOT_VENDAS',
'_6066_1_IMG_INT_PREST_SERV': '1_IMG_INT_PREST_SERV',
'_6066_2_IMG_COM_PREST_SERV': '2_IMG_COM_PREST_SERV',
'_6066_3_IMG_EC_PREST_SERV': '3_IMG_EC_PREST_SERV',
'_6066_4_IMG_TOT_PREST_SERV': '4_IMG_TOT_PREST_SERV',
'_6067_1_IMG_INT_COMPRAS': '1_IMG_INT_COMPRAS',
'_6067_2_IMG_COM_COMPRAS': '2_IMG_COM_COMPRAS',
'_6067_3_IMG_EC_COMPRAS': '3_IMG_EC_COMPRAS',
'_6067_4_IMG_TOT_COMPRAS': '4_IMG_TOT_COMPRAS',
'_6068_1_IMG_INT_FORN_SEREXTERN': '1_IMG_INT_FORN_SEREXTERN',
'_6068_2_IMG_COM_FORN_SEREXTERN': '2_IMG_COM_FORN_SEREXTERN',
'_6068_3_IMG_EC_FORN_SEREXTERN': '3_IMG_EC_FORN_SEREXTERN',
'_6068_4_IMG_TOT_FORN_SEREXTERN': '4_IMG_TOT_FORN_SEREXTERN',
'_6069_1_IMG_INT_AQUIS_ACT_FIX_TANG': '1_IMG_INT_AQUIS_ACT_FIX_TANG',
'_6069_2_IMG_COM_AQUIS_ACT_FIX_TANG': '2_IMG_COM_AQUIS_ACT_FIX_TANG',
'_6069_3_IMG_EC_AQUIS_ACT_FIX_TANG': '3_IMG_EC_AQUIS_ACT_FIX_TANG',
'_6069_4_IMG_TOT_AQUIS_ACT_FIX_TANG': '4_IMG_TOT_AQUIS_ACT_FIX_TANG',
'_6070_1_IMG_INT_AQUIS_PROP_INVEST': '1_IMG_INT_AQUIS_PROP_INVEST',
'_6070_2_IMG_COM_AQUIS_PROP_INVEST': '2_IMG_COM_AQUIS_PROP_INVEST',
'_6070_3_IMG_EC_AQUIS_PROP_INVEST': '3_IMG_EC_AQUIS_PROP_INVEST',
'_6070_4_IMG_TOT_AQUIS_PROP_INVEST': '4_IMG_TOT_AQUIS_PROP_INVEST',
'_6071_1_IMG_INT_AQUIS_ACT_INTANG': '1_IMG_INT_AQUIS_ACT_INTANG',
'_6071_2_IMG_COM_AQUIS_ACT_INTANG': '2_IMG_COM_AQUIS_ACT_INTANG',
'_6071_3_IMG_EC_AQUIS_ACT_INTANG': '3_IMG_EC_AQUIS_ACT_INTANG',
'_6071_4_IMG_TOT_AQUIS_ACT_INTANG': '4_IMG_TOT_AQUIS_ACT_INTANG',
'_6072_1_IMG_INT_REND_SUPLEM': '1_IMG_INT_REND_SUPLEM',
'_6072_2_IMG_COM_REND_SUPLEM': '2_IMG_COM_REND_SUPLEM',
'_6072_3_IMG_EC_REND_SUPLEM': '3_IMG_EC_REND_SUPLEM',
'_6072_4_IMG_TOT_REND_SUPLEM': '4_IMG_TOT_REND_SUPLEM',
'_6073_1_IMG_INT_RS_SERSOCIAIS': '1_IMG_INT_RS_SERSOCIAIS',
'_6073_2_IMG_COM_RS_SERSOCIAIS': '2_IMG_COM_RS_SERSOCIAIS',
'_6073_3_IMG_EC_RS_SERSOCIAIS': '3_IMG_EC_RS_SERSOCIAIS',
'_6073_4_IMG_TOT_RS_SERSOCIAIS': '4_IMG_TOT_RS_SERSOCIAIS',
'_6074_1_IMG_INT_RS_ALUG_EQUIP': '1_IMG_INT_RS_ALUG_EQUIP',
'_6074_2_IMG_COM_RS_ALUG_EQUIP': '2_IMG_COM_RS_ALUG_EQUIP',
'_6074_3_IMG_EC_RS_ALUG_EQUIP': '3_IMG_EC_RS_ALUG_EQUIP',
'_6074_4_IMG_TOT_RS_ALUG_EQUIP': '4_IMG_TOT_RS_ALUG_EQUIP',
'_6075_1_IMG_INT_RS_EST_PROJ_ASS_TEC': '1_IMG_INT_RS_EST_PROJ_ASS_TEC',
'_6075_2_IMG_COM_RS_EST_PROJ_ASS_TEC': '2_IMG_COM_RS_EST_PROJ_ASS_TEC',
'_6075_3_IMG_EC_RS_EST_PROJ_ASS_TEC': '3_IMG_EC_RS_EST_PROJ_ASS_TEC',
'_6075_4_IMG_TOT_RS_EST_PROJ_ASS_TEC': '4_IMG_TOT_RS_EST_PROJ_ASS_TEC',
'_6076_1_IMG_INT_RS_ROYALTIES': '1_IMG_INT_RS_ROYALTIES',
'_6076_2_IMG_COM_RS_ROYALTIES': '2_IMG_COM_RS_ROYALTIES',
'_6076_3_IMG_EC_RS_ROYALTIES': '3_IMG_EC_RS_ROYALTIES',
'_6076_4_IMG_TOT_RS_ROYALTIES': '4_IMG_TOT_RS_ROYALTIES',
'_6077_1_IMG_INT_RS_OUTROS': '1_IMG_INT_RS_OUTROS',
'_6077_2_IMG_COM_RS_OUTROS': '2_IMG_COM_RS_OUTROS',
'_6077_3_IMG_EC_RS_OUTROS': '3_IMG_EC_RS_OUTROS',
'_6077_4_IMG_TOT_RS_OUTROS': '4_IMG_TOT_RS_OUTROS',
'_6078_1_IMG_INT_PM_VEND_PREST_SERV': '1_IMG_INT_PM_VEND_PREST_SERV',
'_6078_2_IMG_COM_PM_VEND_PREST_SERV': '2_IMG_COM_PM_VEND_PREST_SERV',
'_6078_3_IMG_EC_PM_VEND_PREST_SERV': '3_IMG_EC_PM_VEND_PREST_SERV',
'_6078_4_IMG_TOT_PM_VEND_PREST_SERV': '4_IMG_TOT_PM_VEND_PREST_SERV',
'_6079_1_IMG_INT_PM_COMP_FORN_SER_EXT': '1_IMG_INT_PM_COMP_FORN_SER_EXT',
'_6079_2_IMG_COM_PM_COMP_FORN_SER_EXT': '2_IMG_COM_PM_COMP_FORN_SER_EXT',
'_6079_3_IMG_EC_PM_COMP_FORN_SER_EXT': '3_IMG_EC_PM_COMP_FORN_SER_EXT',
'_6079_4_IMG_TOT_PM_COMP_FORN_SER_EXT': '4_IMG_TOT_PM_COMP_FORN_SER_EXT'
}
)

In [14]:
df_ies_sing_renamed = df_ies_sing.rename(
columns={'_6012_1_PESSOAL_NMP_PSERNR': '1_PESSOAL_NMP_PSERNR',
'_6001_VENDAS_SERVICOS_PRESTADOS': 'VENDAS_SERVICOS_PRESTADOS',
'_6002_SUBSIDIOS_EXPLORACAO': 'SUBSIDIOS_EXPLORACAO',
'_6003_GANHOS_PERDAS_SUBSIDIARIAS': 'GANHOS_PERDAS_SUBSIDIARIAS',
'_6004_VARIA_INVENTARIOS_PRODUCAO': 'VARIA_INVENTARIOS_PRODUCAO',
'_6005_TRABALHOS_PROPRIA_ENTIDADE': 'TRABALHOS_PROPRIA_ENTIDADE',
'_6006_CMVMC': 'CMVMC',
'_6007_FSE': 'FSE',
'_6008_GASTOS_PESSOAL': 'GASTOS_PESSOAL',
'_6009_IMPARIDADE_INVENTARIOS': 'IMPARIDADE_INVENTARIOS',
'_6010_IMPARIDADE_DIVIDAS_A_RECEBER': 'IMPARIDADE_DIVIDAS_A_RECEBER',
'_6011_PROVISOES': 'PROVISOES',
'_6012_IMPARIDADE_INVENTA_N_AMORT_INVESTIMENTOS_N_DEPRECIAVEIS': 'IMPARIDADE_INVENTA_N_AMORT_INVESTIMENTOS_N_DEPRECIAVEIS',
'_6013_OUTRAS_IMPARIDADES': 'OUTRAS_IMPARIDADES',
'_6014_AUMENTOS_RED_JUSTO_VALOR': 'AUMENTOS_RED_JUSTO_VALOR',
'_6015_OUTROS_REDIMENTOS_GANHOS': 'OUTROS_REDIMENTOS_GANHOS',
'_6016_OUTROS_GASTOS_PERDAS': 'OUTROS_GASTOS_PERDAS',
'_6017_RES_ANTES_DEPRECIACAO_GASTOS': 'RES_ANTES_DEPRECIACAO_GASTOS',
'_6018_GASTOS_DEPRECIACAO_AMORTIZA': 'GASTOS_DEPRECIACAO_AMORTIZA',
'_6019_IMPARIDADE_INVEST_AMORTIZ_DEPRECIAVEIS': 'IMPARIDADE_INVEST_AMORTIZ_DEPRECIAVEIS',
'_6020_RESULTADO_OPERACIONAL': 'RESULTADO_OPERACIONAL',
'_6021_JUROS_RENDIME_SIMILARES_OBT': 'JUROS_RENDIME_SIMILARES_OBT',
'_6022_JUROS_GAST_SIMILARES_SUPORT': 'JUROS_GAST_SIMILARES_SUPORT',
'_6023_RES_ANTES_IMPOSTOS': 'RES_ANTES_IMPOSTOS',
'_6024_IMPOSTO_RENDIMENTO_PERIODO': 'IMPOSTO_RENDIMENTO_PERIODO',
'_6025_RESULTADO_LIQUIDO_PERIODO': 'RESULTADO_LIQUIDO_PERIODO',
'_6026_resultado_das_atividades_descontinuadas': 'resultado_das_atividades_descontinuadas',
'_6101_ATIVO_NCOR_FIXOS_TANGIVEIS': 'ATIVO_NCOR_FIXOS_TANGIVEIS',
'_6102_ATIVO_NCOR_PROPRI_INVESTIMENTO': 'ATIVO_NCOR_PROPRI_INVESTIMENTO',
'_6103_ATIVO_NCOR_GOODWILL': 'ATIVO_NCOR_GOODWILL',
'_6104_ATIVO_NCOR_INTANGIVEIS': 'ATIVO_NCOR_INTANGIVEIS',
'_6105_ATIVO_NCOR_BIOLOGICOS': 'ATIVO_NCOR_BIOLOGICOS',
'_6106_ATIVO_NCOR_PART_FINAN_EQV_PAT': 'ATIVO_NCOR_PART_FINAN_EQV_PAT',
'_6107_ATIVO_NCOR_PART_FIN_OUTROS_MET': 'ATIVO_NCOR_PART_FIN_OUTROS_MET',
'_6108_ATIVO_NCOR_ACCIONISTAS_SOCIOS': 'ATIVO_NCOR_ACCIONISTAS_SOCIOS',
'_6109_ATIVO_NCOR_OUTROS_ACT_FINANC': 'ATIVO_NCOR_OUTROS_ACT_FINANC',
'_6110_ATIVO_NCOR_IMPOSTOS_DIFERIDOS': 'ATIVO_NCOR_IMPOSTOS_DIFERIDOS',
'_6111_ATIVO_NCOR_INV_FINANC_PQ_ENTID': 'ATIVO_NCOR_INV_FINANC_PQ_ENTID',
'_6112_ATIVO_NCOR_TOTAL': 'ATIVO_NCOR_TOTAL',
'_6113_ATIVO_COR_INVENTARIOS': 'ATIVO_COR_INVENTARIOS',
'_6114_ATIVO_COR_ACTIVOS_BIOLOGICOS': 'ATIVO_COR_ACTIVOS_BIOLOGICOS',
'_6115_ATIVO_COR_CLIENTES': 'ATIVO_COR_CLIENTES',
'_6116_ATIVO_COR_ADIANTAMENTOS_FORNEC': 'ATIVO_COR_ADIANTAMENTOS_FORNEC',
'_6117_ATIVO_COR_ESTADO_OUT_ENTES_PUB': 'ATIVO_COR_ESTADO_OUT_ENTES_PUB',
'_6118_ATIVO_COR_ACCIONISTAS_SOCIOS': 'ATIVO_COR_ACCIONISTAS_SOCIOS',
'_6119_ATIVO_COR_OUTRA_CONT_A_RECEBER': 'ATIVO_COR_OUTRA_CONT_A_RECEBER',
'_6120_ATIVO_COR_DIFERIMENTOS': 'ATIVO_COR_DIFERIMENTOS',
'_6121_ATIVO_COR_ACT_FINAC_DETIDO_NEG': 'ATIVO_COR_ACT_FINAC_DETIDO_NEG',
'_6122_ATIVO_COR_OUTROS_ACT_FINANCEIR': 'ATIVO_COR_OUTROS_ACT_FINANCEIR',
'_6123_ATIVO_COR_ACT_N_COR_DET_VENDA': 'ATIVO_COR_ACT_N_COR_DET_VENDA',
'_6124_ATIVO_COR_OUTROS_ACT_CORRENTES': 'ATIVO_COR_OUTROS_ACT_CORRENTES',
'_6125_ATIVO_COR_CAIXA_DEP_BANCARIOS': 'ATIVO_COR_CAIXA_DEP_BANCARIOS',
'_6126_ATIVO_COR_TOTAL': 'ATIVO_COR_TOTAL',
'_6127_ATIVO_TOTAL': 'ATIVO_TOTAL',
'_6128_CP_CAPITAL_REALIZADO': 'CP_CAPITAL_REALIZADO',
'_6129_CP_ACCOES_PROPRIAS': 'CP_ACCOES_PROPRIAS',
'_6130_CP_OUTROS_INSTR_CAP_PROPRIO': 'CP_OUTROS_INSTR_CAP_PROPRIO',
'_6131_CP_PREMIOS_EMISSAO': 'CP_PREMIOS_EMISSAO',
'_6132_CP_RESERVAS_LEGAIS': 'CP_RESERVAS_LEGAIS',
'_6133_CP_OUTRAS_RESERVAS': 'CP_OUTRAS_RESERVAS',
'_6134_CP_RESULTADOS_TRANSITADOS': 'CP_RESULTADOS_TRANSITADOS',
'_6135_CP_AJUST_EM_ACT_FINANCEIROS': 'CP_AJUST_EM_ACT_FINANCEIROS',
'_6136_CP_EXCENDENTES_REVALORIZACAO': 'CP_EXCENDENTES_REVALORIZACAO',
'_6137_CP_OUTRAS_VARIACAOES_CAP_PRO': 'CP_OUTRAS_VARIACAOES_CAP_PRO',
'_6138_CP_RESULTADO_LIQUIDO_PERIODO': 'CP_RESULTADO_LIQUIDO_PERIODO',
'_6139_CP_TOTAL': 'CP_TOTAL',
'_6140_PASSIVO_NC_PROVISOES': 'PASSIVO_NC_PROVISOES',
'_6141_PASSIVO_NC_FINANCIAMENTOS_OBTD': 'PASSIVO_NC_FINANCIAMENTOS_OBTD',
'_6142_PASSIVO_NC_RESP_BENEF_POS_EMPREG': 'PASSIVO_NC_RESP_BENEF_POS_EMPREG',
'_6143_PASSIVO_NC_IMPOSTOS_DIFERIDOS': 'PASSIVO_NC_IMPOSTOS_DIFERIDOS',
'_6144_PASSIVO_NC_OUTRAS_CONTAS_A_PAGAR': 'PASSIVO_NC_OUTRAS_CONTAS_A_PAGAR',
'_6145_PASSIVO_NC_TOTAL': 'PASSIVO_NC_TOTAL',
'_6146_PASSIVO_COR_FORNCEDORES': 'PASSIVO_COR_FORNCEDORES',
'_6147_PASSIVO_COR_ADIANTA_DE_CLIENTES': 'PASSIVO_COR_ADIANTA_DE_CLIENTES',
'_6148_PASSIVO_COR_ESTADO_OUT_ENT_PUB': 'PASSIVO_COR_ESTADO_OUT_ENT_PUB',
'_6149_PASSIVO_COR_ACCIONISTAS_SOCIOS': 'PASSIVO_COR_ACCIONISTAS_SOCIOS',
'_6150_PASSIVO_COR_FINANCIAMENTOS_OBTD': 'PASSIVO_COR_FINANCIAMENTOS_OBTD',
'_6151_PASSIVO_COR_OUT_CONTAS_A_PAGAR': 'PASSIVO_COR_OUT_CONTAS_A_PAGAR',
'_6152_PASSIVO_COR_DIFERIMENTOS': 'PASSIVO_COR_DIFERIMENTOS',
'_6153_PASSIVO_COR_PASS_FINANC_DET_NEG': 'PASSIVO_COR_PASS_FINANC_DET_NEG',
'_6154_PASSIVO_COR_OUTROS_PASS_FINANCEI': 'PASSIVO_COR_OUTROS_PASS_FINANCEI',
'_6155_PASSIVO_COR_PAS_NC_DETIDOS_VENDA': 'PASSIVO_COR_PAS_NC_DETIDOS_VENDA',
'_6156_PASSIVO_COR_OUTROS_PAS_CORRENTES': 'PASSIVO_COR_OUTROS_PAS_CORRENTES',
'_6157_PASSIVO_COR_TOTAL': 'PASSIVO_COR_TOTAL',
'_6158_PASSIVO_TOTAL': 'PASSIVO_TOTAL',
'_6159_CP_E_PASSIVO_TOTAL': 'CP_E_PASSIVO_TOTAL'}
)

In [15]:
df_ies_sing_renamed.loc[:, 'BENEF_POS_EMPREG'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_RS_EST_PROJ_ASS_TEC'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_PM_VEND_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_AQUIS_ACT_FIX_TANG'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NHT_PSE_TEMPO_PARCIAL'] = None
df_ies_sing_renamed.loc[:, 'GP_REMUN_ORGAOS_SOCIAIS'] = None
df_ies_sing_renamed.loc[:, 'IAE_GASTOS_PESS'] = None
df_ies_sing_renamed.loc[:, '2_N_PESSOAL_NHT_PSETP_REMUNERADAS'] = None
df_ies_sing_renamed.loc[:, 'IAE_AFT_TOTAL_AQUIS'] = None
df_ies_sing_renamed.loc[:, 'CP_SOMA'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_VENDAS'] = None
df_ies_sing_renamed.loc[:, 'IAE_AFT_QUANT_ESCR_LIQ_FIN'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_PM_VEND_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, 'GP_SEG_ACID_TRAB_DOEN_PROF'] = None
df_ies_sing_renamed.loc[:, 'IAE_VENDAS_PAISDRR'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_VENDAS'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_FORN_SEREXTERN'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_PM_COMP_FORN_SER_EXT'] = None
df_ies_sing_renamed.loc[:, 'GP_BPE_OUTROS_BENEF'] = None
df_ies_sing_renamed.loc[:, 'OGP_GASTOS_FORMACAO'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_REND_SUPLEM'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_AQUIS_ACT_INTANG'] = None
df_ies_sing_renamed.loc[:, 'GASTOS_PESSOAL_TOTAL'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_RS_SERSOCIAIS'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_PM_COMP_FORN_SER_EXT'] = None
df_ies_sing_renamed.loc[:, 'IAE_PI_QUANT_ESCR_LIQ_FIN'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_RS_ROYALTIES'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NMP_PSE_HOMENS'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NHT_PESS_REMUN_SE'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NHT_PSE_HOMENS'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_PM_COMP_FORN_SER_EXT'] = None
df_ies_sing_renamed.loc[:, 'IAE_VENDAS'] = None
df_ies_sing_renamed.loc[:, 'GP_BPE_PLAN_CONTRIB_OUTROS'] = None
df_ies_sing_renamed.loc[:, 'IAE_AFT_ADIC_PERIOD_ACT'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_AQUIS_PROP_INVEST'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_AQUIS_PROP_INVEST'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_RS_ROYALTIES'] = None
df_ies_sing_renamed.loc[:, 'IAE_COMPRAS'] = None
df_ies_sing_renamed.loc[:, 'IAE_VENDAS_ACT_BIOL'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_AQUIS_ACT_INTANG'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NHT_PSE_MULHERES'] = None
df_ies_sing_renamed.loc[:, 'GP_GASTOS_ACCAO_SOCIAL'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NHT_PSETC_REMUNERADAS'] = None
df_ies_sing_renamed.loc[:, 'IAE_CMVMC_ACT_BIOL'] = None
df_ies_sing_renamed.loc[:, 'IAE_AFT_TOTAL_AQUIS_EDIF'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_COMPRAS'] = None
df_ies_sing_renamed.loc[:, 'PESSOAL_NHT_PESS_AGENC_TEMPOR'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_PM_VEND_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, 'IAE_PI_TOTAL_AQUIS'] = None
df_ies_sing_renamed.loc[:, 'PESSOAL_NMP_PSE_INVEST_DESENV'] = None
df_ies_sing_renamed.loc[:, 'OGP_GASTOS_FARDAMENTO'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_VENDAS'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_AQUIS_ACT_INTANG'] = None
df_ies_sing_renamed.loc[:, 'IAE_CMVMC'] = None
df_ies_sing_renamed.loc[:, 'IAE_GASTOS_PESS_OUTROS'] = None
df_ies_sing_renamed.loc[:, 'GP_BPE_PREMIOS_PENSOES'] = None
df_ies_sing_renamed.loc[:, 'GP_ENCARG_REMUN'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_RS_ALUG_EQUIP'] = None
df_ies_sing_renamed.loc[:, 'CP_DIVIDENDOS_ANTECIPADOS'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NMP_PSE_TEMPO_COMPLETO'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, 'IAE_PI_TOTAL_AQUIS_EDIF'] = None
df_ies_sing_renamed.loc[:, 'IAE_VENDAS_MERCADORIAS'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NMP_PRESTAD_SERVICOS'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NMP_PSETP_REMUNERADAS'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_RS_SERSOCIAIS'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_COMPRAS'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_RS_SERSOCIAIS'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_RS_OUTROS'] = None
df_ies_sing_renamed.loc[:, 'GP_INDEMINIZACOES'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NMP_PESS_N_REMUN'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_RS_EST_PROJ_ASS_TEC'] = None
df_ies_sing_renamed.loc[:, 'IAE_PI_ADIC_PERIO_PROP_INV'] = None
df_ies_sing_renamed.loc[:, 'GP_REMUN_PESSOAL'] = None
df_ies_sing_renamed.loc[:, 'IAE_GASTOS_PESS_REMUN'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_RS_ALUG_EQUIP'] = None
df_ies_sing_renamed.loc[:, 'NIF_anon'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_FORN_SEREXTERN'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NHT_PRESTAD_SERVICOS'] = None
df_ies_sing_renamed.loc[:, 'IAE_NUM_MED_PESS_SERV'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_RS_SERSOCIAIS'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_AQUIS_ACT_FIX_TANG'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_RS_ROYALTIES'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_AQUIS_ACT_INTANG'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, 'IAE_VAR_INVENT_PROD'] = None
df_ies_sing_renamed.loc[:, 'GP_ROS_PARTIC_LUCROS'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NHT_PESS_N_REMUN'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_RS_ALUG_EQUIP'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_COMPRAS'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NHT_PSERNR'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_RS_OUTROS'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_RS_ALUG_EQUIP'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_RS_EST_PROJ_ASS_TEC'] = None
df_ies_sing_renamed.loc[:, 'GP_BPE_PLAN_CONTRIB_ORG_SOC'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_FORN_SEREXTERN'] = None
df_ies_sing_renamed.loc[:, 'GP_RP_PARTIC_LUCROS'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_AQUIS_PROP_INVEST'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_RS_OUTROS'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_PM_COMP_FORN_SER_EXT'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_REND_SUPLEM'] = None
df_ies_sing_renamed.loc[:, 'IAE_FORN_SERV_EXTER'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_RS_EST_PROJ_ASS_TEC'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_PM_VEND_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_VENDAS'] = None
df_ies_sing_renamed.loc[:, '2_IMG_COM_RS_ROYALTIES'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_REND_SUPLEM'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NMP_PESS_REMUN_SE'] = None
df_ies_sing_renamed.loc[:, '2_PESSOAL_NHT_PSE_TEMPO_COMPLETO'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_AQUIS_ACT_FIX_TANG'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_AQUIS_ACT_FIX_TANG'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_COMPRAS'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NMP_PSE_TEMPO_PARCIAL'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_RS_OUTROS'] = None
df_ies_sing_renamed.loc[:, 'IAE_CMVMC_MATER_PRIMAS'] = None
df_ies_sing_renamed.loc[:, '1_IMG_INT_AQUIS_PROP_INVEST'] = None
df_ies_sing_renamed.loc[:, 'IAE_PREST_SERV'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NMP_PSETC_REMUNERADAS'] = None
df_ies_sing_renamed.loc[:, '3_IMG_EC_REND_SUPLEM'] = None
df_ies_sing_renamed.loc[:, 'IAE_CMVMC_MERCADORIAS'] = None
df_ies_sing_renamed.loc[:, '4_IMG_TOT_FORN_SEREXTERN'] = None
df_ies_sing_renamed.loc[:, '1_PESSOAL_NMP_PSE_MULHERES'] = None
df_ies_sing_renamed.loc[:, 'OUTROS_GASTOS_PESSOAL'] = None

C:\Users\afons\AppData\Local\Temp\ipykernel_19692\3880077222.py:98: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ies_sing_renamed.loc[:, '2_IMG_COM_FORN_SEREXTERN'] = None
C:\Users\afons\AppData\Local\Temp\ipykernel_19692\3880077222.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ies_sing_renamed.loc[:, 'GP_RP_PARTIC_LUCROS'] = None
C:\Users\afons\AppData\Local\Temp\ipykernel_19692\3880077222.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [16]:
df_ies_sing_renamed['rank'] = df_ies_sing_renamed.groupby(['N_Proj_anon', 'ANO_EXERCICIO'])['DATA_RECEPCAO'].rank(method='first', ascending=False)
df_ies_sing_renamed['rank'] = pd.to_numeric(df_ies_sing_renamed['rank'], downcast='integer')
df_ies_sing_renamed_clean = df_ies_sing_renamed[df_ies_sing_renamed['rank']==1]

C:\Users\afons\AppData\Local\Temp\ipykernel_19692\3572414031.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ies_sing_renamed['rank'] = df_ies_sing_renamed.groupby(['N_Proj_anon', 'ANO_EXERCICIO'])['DATA_RECEPCAO'].rank(method='first', ascending=False)


In [17]:
df_ies_sing_renamed_clean['Dimensao_Empresa'] = 'Singular'
df_ies_candidaturas_renamed['Dimensao_Empresa'] = 'Coletivo'

C:\Users\afons\AppData\Local\Temp\ipykernel_19692\1093353557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ies_sing_renamed_clean['Dimensao_Empresa'] = 'Singular'


In [18]:
df_ies_merged = pd.concat([df_ies_candidaturas_renamed, df_ies_sing_renamed_clean], ignore_index=True)
df_ies_merged

,NIF_anon,ANO_EXERCICIO,DATA_RECEPCAO,VENDAS_SERVICOS_PRESTADOS,SUBSIDIOS_EXPLORACAO,GANHOS_PERDAS_SUBSIDIARIAS,VARIA_INVENTARIOS_PRODUCAO,TRABALHOS_PROPRIA_ENTIDADE,CMVMC,FSE,...,Industria40/Tecn_Ii_4,Industria40/Tecn_Iii_4,Uploads/Aplicavel_12,Uploads/Upload_12,Industria40/Tecn_I_6,Industria40/Outra_Tecn_I,Uploads/Aplicavel_08,Uploads/Upload_08,Dimensao_Empresa,resultado_das_atividades_descontinuadas
0,533683393,2013,35:43.0,3575292.13,5899.36,0.0,9406.38,2060.62,1214731.77,888963.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coletivo,NaN
1,533683393,2014,27:06.0,6136916.06,2812.45,0.0,19587.83,4197.72,2268654.92,1972050.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coletivo,NaN
2,533683393,2015,22:53.0,7176441.46,0.00,0.0,9806.96,10934.50,2741715.01,2269277.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coletivo,NaN
3,533683393,2016,11:15.0,7565329.52,0.00,0.0,17359.80,1585.60,3454351.06,1831458.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coletivo,NaN
4,533683393,2017,44:57.0,7960571.39,0.00,0.0,29036.97,756.97,3372989.75,1944322.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coletivo,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44801,None,2019,53:34.0,0.00,0.00,-18750.0,0.00,165901.09,29760.15,44537.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Singular,1284.26
44802,None,2020,51:09.0,0.00,0.00,11500.0,0.00,58784.93,40229.95,28951.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Singular,788.39
44803,None,2013,13:14.0,0.00,0.00,0.0,0.00,0.00,447.82,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Singular,231.07
44804,None,2019,18:55.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Singular,0.00


In [19]:
check_project_year_candidate_duplicates(df_ies_merged)

,N_Proj_anon,ANO_EXERCICIO,size


In [20]:
print("CONCAT IES SING / PROM")
print("Existe projetos a null: ", df_ies_merged['N_Proj_anon'].isnull().values.any())
print("Nr de projetos a null: ", df_ies_merged['N_Proj_anon'].isnull().sum())

CONCAT IES SING / PROM
Existe projetos a null:  False
Nr de projetos a null:  0


### Obter projetos anulados + terminados

In [21]:
#df_projetos_anulados = df_anulacoes.loc[df_anulacoes['Tipo'] == "Anulação pós-contrato", ['Nproj_anon', 'Motivo']]

In [22]:
df_projetos_anulados = df_anulacoes.loc[df_anulacoes['Tipo'] == "Anulação pós-contrato", ['Nproj_anon','Motivo']]

In [23]:
df_projetos_anulados.drop_duplicates()

,Nproj_anon,Motivo
0,15414,Incumprimento das obrigações ou resultados con...
1,15972,NaN
2,17832,NaN
3,18762,Desistência do promotor
4,19382,NaN
...,...,...
743,5579852,Desistência do promotor
744,5580720,Desistência do promotor
755,5593988,Desistência do promotor
768,5599878,Desistência do promotor


In [24]:
df_teste = df_projetos_anulados.groupby('Nproj_anon').size().to_frame('size')

In [25]:
df_teste[df_teste['size'] > 1]

,size
Nproj_anon,
578095,2
1443894,2


In [26]:
df_projetos_anulados[(df_projetos_anulados['Nproj_anon'] == 578095) | (df_projetos_anulados['Nproj_anon'] == 1443894)]

,Nproj_anon,Motivo
196,578095,NaN
197,578095,Desistência do promotor
630,1443894,Inexecução integral da candidatura
631,1443894,Desistência do promotor


In [27]:
"""
Os seguintes projetos: 578095 e 1443894 não possuem somente um motivo. Ambos passam a ter apenas o motivo da anulação por "Desistência do promotor"
"""
df_projetos_anulados.loc[df_projetos_anulados['Nproj_anon'] == 578095, ['Motivo']] = "Desistência do promotor"
df_projetos_anulados.loc[df_projetos_anulados['Nproj_anon'] == 1443894, ['Motivo']] = "Desistência do promotor"

In [28]:
df_projetos_anulados.drop_duplicates()

,Nproj_anon,Motivo
0,15414,Incumprimento das obrigações ou resultados con...
1,15972,NaN
2,17832,NaN
3,18762,Desistência do promotor
4,19382,NaN
...,...,...
743,5579852,Desistência do promotor
744,5580720,Desistência do promotor
755,5593988,Desistência do promotor
768,5599878,Desistência do promotor


In [29]:
df_projetos_anulados['Anulado'] = 1

In [30]:
# OR Condition para obter um maior número de projetos terminados
df_projetos_terminados = df_facie_appie.groupby('Nproj_anon').agg({'Tipo':lambda x: ','.join(x)})
df_projetos_terminados2 = df_projetos_terminados[(df_projetos_terminados['Tipo'].str.contains("ENCPRJ", case=False)) | (df_projetos_terminados['Tipo'].str.contains("ENCINV", case=False))]
df_projetos_terminados2.reset_index(inplace=True)

In [31]:
df_projetos_terminados2['Terminado'] = 1

C:\Users\afons\AppData\Local\Temp\ipykernel_19692\3578149585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_projetos_terminados2['Terminado'] = 1


In [32]:
df_projetos_terminados2

,Nproj_anon,Tipo,Terminado
0,15166,"ENCPRJ,ENCINV,PPI,PPI",1
1,15197,"ENCPRJ,ENCINV,PPI",1
2,15383,"ENCINV,PPI,PPI,PPI,PPI",1
3,15662,"ENCINV,PPI,PPI,PPI,PPI,PPI,PPI,PPI",1
4,15910,"ENCINV,PPI,PPI",1
...,...,...,...
802,1240193,"ENCINV,PPI,PPI",1
803,1240751,"ENCINV,PPI,PPI,PPI,PPI,PPI",1
804,1254577,"ENCINV,PPI,PPI,PPI,PPI,PPI,PPI",1
805,1254794,"ENCINV,PPI,PPI,PPI",1


In [33]:
print('Numero de projetos terminados unicos: ' + str(len(df_projetos_terminados2['Nproj_anon'].unique())))

Numero de projetos terminados unicos: 807


In [34]:
# Obter projetos anulados
df_final = df_ies_merged.merge(df_projetos_anulados.drop_duplicates(), left_on="N_Proj_anon", right_on="Nproj_anon", how="left")

In [35]:
# Obter projetos terminados
df_final2 = df_final.merge(df_projetos_terminados2.drop_duplicates(), left_on="N_Proj_anon", right_on="Nproj_anon", how="left")

In [36]:
df_ies_merged.shape

(44806, 405)

In [37]:
df_final.shape

(44806, 408)

In [38]:
df_final2.shape

(44806, 411)

In [39]:
df_final2

,NIF_anon,ANO_EXERCICIO,DATA_RECEPCAO,VENDAS_SERVICOS_PRESTADOS,SUBSIDIOS_EXPLORACAO,GANHOS_PERDAS_SUBSIDIARIAS,VARIA_INVENTARIOS_PRODUCAO,TRABALHOS_PROPRIA_ENTIDADE,CMVMC,FSE,...,Uploads/Aplicavel_08,Uploads/Upload_08,Dimensao_Empresa,resultado_das_atividades_descontinuadas,Nproj_anon_x,Motivo,Anulado,Nproj_anon_y,Tipo,Terminado
0,533683393,2013,35:43.0,3575292.13,5899.36,0.0,9406.38,2060.62,1214731.77,888963.00,...,NaN,NaN,Coletivo,NaN,NaN,NaN,NaN,21149.0,"ENCINV,PPI,PPI,PPI",1.0
1,533683393,2014,27:06.0,6136916.06,2812.45,0.0,19587.83,4197.72,2268654.92,1972050.22,...,NaN,NaN,Coletivo,NaN,NaN,NaN,NaN,21149.0,"ENCINV,PPI,PPI,PPI",1.0
2,533683393,2015,22:53.0,7176441.46,0.00,0.0,9806.96,10934.50,2741715.01,2269277.86,...,NaN,NaN,Coletivo,NaN,NaN,NaN,NaN,21149.0,"ENCINV,PPI,PPI,PPI",1.0
3,533683393,2016,11:15.0,7565329.52,0.00,0.0,17359.80,1585.60,3454351.06,1831458.57,...,NaN,NaN,Coletivo,NaN,NaN,NaN,NaN,21149.0,"ENCINV,PPI,PPI,PPI",1.0
4,533683393,2017,44:57.0,7960571.39,0.00,0.0,29036.97,756.97,3372989.75,1944322.79,...,NaN,NaN,Coletivo,NaN,NaN,NaN,NaN,21149.0,"ENCINV,PPI,PPI,PPI",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44801,None,2019,53:34.0,0.00,0.00,-18750.0,0.00,165901.09,29760.15,44537.58,...,NaN,NaN,Singular,1284.26,NaN,NaN,NaN,NaN,NaN,NaN
44802,None,2020,51:09.0,0.00,0.00,11500.0,0.00,58784.93,40229.95,28951.70,...,NaN,NaN,Singular,788.39,NaN,NaN,NaN,NaN,NaN,NaN
44803,None,2013,13:14.0,0.00,0.00,0.0,0.00,0.00,447.82,0.00,...,NaN,NaN,Singular,231.07,NaN,NaN,NaN,NaN,NaN,NaN
44804,None,2019,18:55.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,NaN,NaN,Singular,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_final2.to_csv(path_save + 'finaldataset.csv', index=False, encoding='latin-1')